# gsim.meep — MEEP Photonic FDTD Simulation

This notebook demonstrates the `gsim.meep` module workflow:

1. Create a gdsfactory component
2. Configure simulation with **declarative API** (typed physics objects)
3. Validate config
4. **Visualize 3D geometry** (client-side, no MEEP needed)
5. Write config (GDS + JSON + runner script) to disk
6. Inspect the generated files
7. **Server-side diagnostics** (geometry/field PNGs from MEEP runner)

> **Note:** MEEP runs on the cloud — no local MEEP installation needed.  
> Geometry is sent as a **GDS file** alongside a JSON config with layer stack info.

### Performance features

| Feature | Speedup | API |
|---------|---------|-----|
| DFT-convergence stopping | best for S-params | `sim.solver.stopping = meep.DFTDecay(...)` |
| Field-decay stopping | 1.5-3x | `sim.solver.stopping = meep.FieldDecay(...)` |
| Wide source bandwidth | edge-freq accuracy | `sim.source = meep.ModeSource()` (auto ~3x monitor bw) |
| Polygon simplification | 10-100x fewer vertices | `sim.solver.simplify_tol = 0.01` |
| Subpixel averaging control | variable | `sim.solver.subpixel = True` |
| Verbose progress stepping | observability | `sim.diagnostics.verbose_interval = 5.0` |
| Eigenmode debug logging | diagnostics | Auto-saved `meep_debug.json` (n_eff, power conservation) |
| Server-side diagnostics | geometry validation | `sim.diagnostics.save_geometry = True` |
| Preview-only mode | seconds vs minutes | `sim.diagnostics.preview_only = True` |
| Reduced default margins | 1.3-2x | Default `margin/margin_z` = 0.5 (was 1.0) |
| Fewer frequency points | ~2x | Default `num_freqs` = 11 (was 21) |
| Port extension into PML | accuracy | Auto (default) via `sim.domain.extend_ports` |

> **Note on symmetry:** `mp.Mirror` symmetries are **not used** for S-parameter extraction — MEEP's eigenmode coefficient normalization is incorrect when source monitors straddle a symmetry plane. This matches gplugins' approach. Symmetries are only applied in preview-only mode.

In [ ]:
import gdsfactory as gf
from ubcpdk import PDK, cells

PDK.activate()

## 1. Create a component

In [ ]:
component = cells.ebeam_y_1550()
component.plot()

In [ ]:
# Inspect ports
for p in component.ports:
    print(f"{p.name}: center={p.center}, width={p.width}, orientation={p.orientation}")

## 2. Configure Simulation (declarative API)

In [ ]:
from gsim import meep

sim = meep.Simulation()

# Set the component geometry
sim.geometry.component = component

# Set layer stack (from active PDK) with auto z-crop
sim.geometry.z_crop = "auto"

# Override material optical properties
sim.materials = {"si": 3.47, "SiO2": 1.44}

# Configure wavelength range via monitors
sim.monitors = [
    meep.ModeMonitor(port="o1", wavelength=1.55, bandwidth=0.01, num_freqs=11),
    meep.ModeMonitor(port="o2", wavelength=1.55, bandwidth=0.01, num_freqs=11),
    meep.ModeMonitor(port="o3", wavelength=1.55, bandwidth=0.01, num_freqs=11),
]

# Source configuration: auto bandwidth (~3x monitor bw) ensures
# edge frequencies receive adequate spectral power
sim.source = meep.ModeSource()  # auto fwidth, auto port

# Domain: margins + PML (defaults: margin=0.5, pml=1.0)
# Waveguide ports are auto-extended into PML (extend_ports=0 → auto = margin + pml)
sim.domain = meep.Domain(margin=0.5)

# Solver: resolution, stopping, accuracy
# DFT convergence stopping — best for S-params, with 200 time unit cap
# min_time=100 prevents false convergence before pulse traverses the device
sim.solver = meep.FDTD(
    resolution=20,  # 20 pixels/um — fast for iteration; use 30-40+ for production
    stopping=meep.DFTDecay(max_time=200, threshold=1e-3, min_time=100),
    simplify_tol=0.01,  # simplify dense GDS polygons (10nm tolerance)
    subpixel=True,
)

# Note: mp.Mirror symmetries are NOT used for S-parameter extraction
# (causes incorrect eigenmode normalization). This matches gplugins' approach.

# Server-side diagnostics: geometry cross-section PNGs + field snapshot + animation
# Set preview_only=True to skip FDTD and just validate geometry (seconds)
sim.diagnostics = meep.Diagnostics(
    save_geometry=True, save_fields=True, save_animation=True, verbose_interval=5.0
)

# Output directory
sim.output_dir = "./meep-sim-test"

## 3. Validate configuration

In [ ]:
result = sim.validate_config()
print(result)
print(f"Valid: {result.valid}")
if result.errors:
    print("Errors:", result.errors)
if result.warnings:
    print("Warnings:", result.warnings)

## 4. Inspect config models

In [ ]:
# Wavelength config — derived from monitors
wl = sim._wavelength_config()
print(f"Wavelength: {wl.wavelength} um")
print(f"Bandwidth:  {wl.bandwidth} um")
print(f"fcen:       {wl.fcen:.4f} (1/um)")
print(f"df:         {wl.df:.4f} (1/um)")
print(f"Num freqs:  {wl.num_freqs}")

# Source config
src = sim._source_config()
src_fwidth = src.compute_fwidth(wl.fcen, wl.df)
print(f"\nSource config:")
print(f"  bandwidth: {sim.source.bandwidth} (None=auto)")
print(f"  port:      {sim.source.port} (None=auto)")
print(f"  fwidth:    {src_fwidth:.4f} (1/um) — vs monitor df={wl.df:.4f}")

# Stopping config
stopping = sim.solver.stopping
print(f"\nStopping: {type(stopping).__name__}")
print(f"  max_time: {stopping.max_time}")
if isinstance(stopping, meep.FieldDecay):
    print(f"  threshold:  {stopping.threshold}")
    print(f"  dt:         {stopping.dt}")
    print(f"  component:  {stopping.component}")
elif isinstance(stopping, meep.DFTDecay):
    print(f"  threshold:  {stopping.threshold}")
    print(f"  min_time:   {stopping.min_time}")

# Symmetries
print(f"\nSymmetries: {len(sim.symmetries)}")
for s in sim.symmetries:
    print(f"  Mirror {s.direction}, phase={s.phase}")

# Domain config
dom = sim.domain
print(f"\nDomain config:")
print(f"  pml:           {dom.pml} um")
print(f"  margin:        {dom.margin} um")
print(f"  port_margin:   {dom.port_margin} um")
print(f"  extend_ports:  {dom.extend_ports} (0=auto: margin + pml = {dom.margin + dom.pml} um)")

# Solver config
print(f"\nSolver config:")
print(f"  resolution:       {sim.solver.resolution} pixels/um")
print(f"  subpixel:         {sim.solver.subpixel}")
print(f"  simplify_tol:     {sim.solver.simplify_tol} um")

# Diagnostics config
diag = sim.diagnostics
print(f"\nDiagnostics config:")
print(f"  save_geometry:     {diag.save_geometry}")
print(f"  save_fields:       {diag.save_fields}")
print(f"  save_epsilon_raw:  {diag.save_epsilon_raw}")
print(f"  preview_only:      {diag.preview_only}")
print(f"  verbose_interval:  {diag.verbose_interval} MEEP time units")

In [ ]:
# Resolution
print(f"Resolution: {sim.solver.resolution} pixels/um")

# Resolution presets (from legacy config)
from gsim.meep import ResolutionConfig

for name in ["coarse", "default", "fine"]:
    preset = getattr(ResolutionConfig, name)()
    print(f"  {name}: {preset.pixels_per_um} pixels/um")

## 5. Visualize geometry with simulation overlay

The `plot_2d()` and `plot_3d()` methods build a `GeometryModel` from the component + stack and render using solver-agnostic code.

When the stack and ports are configured, `plot_2d()` also draws:
- **Sim cell boundary** (dashed black) — geometry bbox + PML + margin
- **PML regions** — semi-transparent orange shading at cell edges
- **Source port** — red line with arrow showing excitation direction
- **Monitor ports** — blue lines at monitor locations

## 5. Visualize 3D geometry (client-side, no MEEP)

The `plot_2d()` and `plot_3d()` methods build a `GeometryModel` from the component + stack and render using solver-agnostic code.

In [ ]:
# 2D cross-section at z = center of core layer
sim.plot_2d(slices="z")

In [ ]:
# Multi-view: x, y, z cross-sections
sim.plot_2d(slices="xyz")

## 6. Write config to disk

`write_config()` generates three files:
- `layout.gds` — the GDS layout
- `sim_config.json` — layer stack, ports, materials, FDTD params
- `run_meep.py` — self-contained cloud runner script

In [ ]:
output_dir = sim.write_config()
print(f"Config written to: {output_dir}")

# List generated files
import os

for f in sorted(os.listdir(output_dir)):
    size = os.path.getsize(output_dir / f)
    print(f"  {f} ({size:,} bytes)")

## 9. Parse S-parameter results

In production, `sim.simulate()` sends the config to the cloud and returns parsed results.  
Here we demo the result parsing from CSV data.

The runner also saves `meep_debug.json` alongside `s_parameters.csv` with eigenmode diagnostics (n_eff, kdom, power conservation). This is auto-loaded into `result.debug_info` when parsing results.

In [ ]:
from pathlib import Path

import numpy as np
from gsim.meep import SParameterResult

output_dir = Path("./meep-sim-test")
csv_data = output_dir / "s_parameters.csv"

# Parse results (auto-loads meep_debug.json if present)
result = SParameterResult.from_csv(csv_data)
print(f"Wavelengths: {len(result.wavelengths)} points")
print(f"S-params: {list(result.s_params.keys())}")
print(f"Port names: {result.port_names}")
print(f"Debug info loaded: {bool(result.debug_info)}")

# After a cloud run, debug_info contains eigenmode diagnostics:
if result.debug_info:
    meta = result.debug_info.get("metadata", {})
    print(f"\nSimulation metadata:")
    print(f"  Resolution: {meta.get('resolution')} pixels/um")
    print(f"  Cell size: {meta.get('cell_size')}")
    print(f"  Wall time: {meta.get('wall_seconds', 0):.1f}s")
    print(f"  Stopping mode: {meta.get('stopping_mode')}")

    # Check eigenmode n_eff — should be ~2.44 for Si waveguide at 1550nm
    for port, info in result.debug_info.get("eigenmode_info", {}).items():
        n_effs = info.get("n_eff", [])
        if n_effs:
            print(f"  Port {port} n_eff (center): {n_effs[len(n_effs)//2]:.3f}")

    # Power conservation — should be ~1.0
    pcons = result.debug_info.get("power_conservation", [])
    if pcons:
        print(f"  Power conservation (center): {pcons[len(pcons)//2]:.3f}")

In [ ]:
# Plot S-parameters (dB scale)
fig = result.plot(db=True)

## 9b. Server-side diagnostics

After a cloud run, the MEEP runner saves geometry cross-section PNGs and field snapshots alongside S-parameters. These are auto-detected and loaded into `result.diagnostic_images`.

| File | When | Content |
|------|------|---------|
| `meep_geometry_xy.png` | Pre-run (default) | XY cross-section: epsilon + sources + monitors + PML |
| `meep_geometry_xz.png` | Pre-run (3D only) | XZ cross-section: layer stack side view |
| `meep_geometry_yz.png` | Pre-run (3D only) | YZ cross-section: port face view |
| `meep_fields_xy.png` | Post-run (default) | Ey field overlaid on epsilon |

For **preview-only** mode (geometry validation without running FDTD), use `SParameterResult.from_directory()` since there's no CSV.

In [ ]:
# Display diagnostic images inline (after a cloud run)
# result.show_diagnostics()

# For full run: loads from CSV + auto-detects PNGs
# result = SParameterResult.from_csv("meep-sim-test/s_parameters.csv")
# result.show_diagnostics()

# For preview-only (no CSV): loads debug JSON + geometry PNGs
# result = SParameterResult.from_directory("meep-sim-test/")
# result.show_diagnostics()

# Check what diagnostic images are available
print(f"Diagnostic images: {list(result.diagnostic_images.keys())}")
for name, path in sorted(result.diagnostic_images.items()):
    print(f"  {name}: {path}")

## 10. Full workflow summary

```python
from gsim import meep

sim = meep.Simulation()
sim.geometry.component = component
sim.geometry.z_crop = "auto"              # auto z-crop around highest-n layer

sim.materials = {"si": 3.47, "SiO2": 1.44}

sim.monitors = [
    meep.ModeMonitor(port="o1", wavelength=1.55, bandwidth=0.1),
    meep.ModeMonitor(port="o2", wavelength=1.55, bandwidth=0.1),
]
sim.source = meep.ModeSource()            # auto fwidth (~3x monitor bw), auto port

sim.domain = meep.Domain(margin=0.5, pml=1.0)  # 0.5um margins, 1um PML

sim.solver = meep.FDTD(
    resolution=32,
    stopping=meep.DFTDecay(               # DFT convergence (best for S-params)
        max_time=200,
        threshold=1e-3,
    ),
    simplify_tol=0.01,                    # simplify dense GDS polygons (10nm)
)

sim.diagnostics = meep.Diagnostics(
    save_geometry=True,
    save_fields=True,
    verbose_interval=5.0,                 # progress prints every 5 MEEP time units
)

sim.output_dir = "./meep-sim-test"

# Visualize before running (client-side: PML, ports, sim cell, dielectrics)
sim.plot_2d(slices="xyz")

# Run on GDSFactory+ cloud (requires auth)
result = sim.run()
result.plot()

# View server-side diagnostics (geometry + field PNGs from MEEP)
result.show_diagnostics()

# Check eigenmode diagnostics (auto-loaded from meep_debug.json)
if result.debug_info:
    for port, info in result.debug_info["eigenmode_info"].items():
        print(f"{port}: n_eff={info['n_eff'][len(info['n_eff'])//2]:.3f}")
    pcons = result.debug_info["power_conservation"]
    print(f"Power conservation: {pcons[len(pcons)//2]:.3f}")
```

### Port extension into PML

Waveguide ports are automatically extended through `margin + pml` into PML at `write_config()` time. This prevents spurious reflections from abrupt waveguide termination. To customize:

```python
sim.domain.extend_ports = 2.0  # explicit 2um extension
sim.domain.extend_ports = 0.0  # auto (default): margin + pml
```

### Preview-only mode (fast geometry validation)

```python
sim.diagnostics.preview_only = True
sim.write_config()
result = sim.run()
result.show_diagnostics()  # geometry PNGs only — no fields, no S-params
```

### Stopping modes

| Mode | API | Best for |
|------|-----|----------|
| Fixed time | `meep.FixedTime(max_time=100)` (default) | Known-duration sims |
| Field decay | `meep.FieldDecay(threshold=1e-3)` | Point monitoring + time cap |
| DFT convergence | `meep.DFTDecay(threshold=1e-3)` | S-parameter extraction (recommended) |

### Source bandwidth

The source Gaussian `fwidth` is decoupled from the monitor frequency span. By default, `ModeSource()` auto-computes `fwidth = max(3*monitor_df, 0.2*fcen)`, ensuring edge frequencies have adequate spectral power. To set explicitly:

```python
sim.source = meep.ModeSource(bandwidth=0.3)  # 0.3 um wavelength bandwidth
sim.source = meep.ModeSource(port="o1")      # explicit source port
```

### Note on symmetry

`mp.Mirror` symmetries are **not used** for S-parameter extraction. MEEP's `get_eigenmode_coefficients` with `add_mode_monitor` produces incorrect normalization when the source monitor straddles a symmetry plane (~2x coefficient error). This matches gplugins, which also never uses `mp.Mirror` for S-parameter runs. Symmetries are only applied in preview-only mode for fast geometry validation.

## 10b. Minimal (all defaults — 0.5um margins, 11 freqs, fixed stopping, auto port extension)

```python
from gsim import meep

sim = meep.Simulation()
sim.geometry.component = component
sim.geometry.z_crop = "auto"
sim.materials = {"si": 3.47}
sim.monitors = [
    meep.ModeMonitor(port="o1", wavelength=1.55, bandwidth=0.1),
    meep.ModeMonitor(port="o2", wavelength=1.55, bandwidth=0.1),
]
sim.output_dir = "./meep-sim-test"

# Optional: speed up dense GDS components
# sim.solver.simplify_tol = 0.01

# Optional: wider source bandwidth + DFT stopping
# sim.source = meep.ModeSource()  # auto ~3x monitor bw
# sim.solver.stopping = meep.DFTDecay(max_time=200)

# Visualize before running
sim.plot_2d(slices="z")

# Run on GDSFactory+ cloud (requires auth)
result = sim.run()
result.plot()
```